In [28]:
# pip install pandas numpy matplotlib statsmodels scikit-learn
import pandas as pd, numpy as np, matplotlib.pyplot as plt, math
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

df = pd.read_csv("https://raw.githubusercontent.com/Laginho/SmartCity-Simulator/refs/heads/main/src/Input/dados_onibus.csv")

# cria ordem de leitura
df["row_id"] = np.arange(len(df))
weekday_order = ["Seg","Ter","Qua","Qui","Sex","Sab","Dom"]
weekday_to_idx = {d:i for i,d in enumerate(weekday_order)}
df["weekday_idx"] = df["dia"].map(weekday_to_idx)

# ordena por linha e ordem de aparecimento
df = df.sort_values(["linha","row_id"]).reset_index(drop=True)

# contador sequencial de dias para cada linha, o eixo temporal do modelo.
df["t_seq"] = df.groupby("linha").cumcount()

capacidade_onibus = 50
resultados, previsoes = {}, []

for linha, g in df.groupby("linha"):
    # Para cada linha de ônibus, cria uma série temporal s indexada por t_seq
    s = g.set_index("t_seq")["requests"]

    # SARIMAX semanal
    model = SARIMAX(s, order=(1,1,2), seasonal_order=(1,1,0,7),
                    enforce_stationarity=False, enforce_invertibility=False)
    res = model.fit(disp=False)

    # previsão 1 passo à frente
    fc = res.get_forecast(steps=1)
    mean = float(fc.predicted_mean.iloc[0])
    lo, hi = fc.conf_int().iloc[0].tolist() #limites inferior e superior do intervalo de confiança

    # erro de treino
    fitted = res.fittedvalues.dropna()
    y_true = s.loc[fitted.index]
    mse = mean_squared_error(y_true, fitted) #Compara com os valores reais da série e mede o erro quadrático médio (MSE)

    prev_round = int(round(mean))
    buses = math.ceil(max(prev_round,0) / capacidade_onibus)

    resultados[linha] = dict(prev=prev_round, buses=buses, mse=mse, lo=lo, hi=hi, last=s.iloc[-1])
    previsoes.append({"linha": linha, "next_day": int(s.index[-1])+1, "requests": prev_round})

# DataFrame final
df_prev = pd.DataFrame(previsoes).sort_values("linha")

# Resumo por linha
for linha, r in resultados.items():
    print(f"Linha {linha}: prev={r['prev']} ônibus={r['buses']}  MSE={r['mse']:.2f}  "
          f"(Intervalo≈[{r['lo']:.1f},{r['hi']:.1f}])")


Linha 1: prev=586 ônibus=12  MSE=340980.23  (Intervalo≈[-142.4,1314.1])
Linha 2: prev=826 ônibus=17  MSE=376553.70  (Intervalo≈[334.7,1317.1])
Linha 3: prev=208 ônibus=5  MSE=78311.84  (Intervalo≈[-97.1,513.7])
Linha 4: prev=279 ônibus=6  MSE=59906.47  (Intervalo≈[62.0,496.8])
Linha 5: prev=59 ônibus=2  MSE=82278.87  (Intervalo≈[-256.3,374.2])
